# Escenarios de precificación de créditos en cartera

Cuaderno para los escenarios de precificación de créditos en cartera. Se proponen tres escenarios: usando optimizador de cartera, aumento constante y aumento por segmentación de cartera.

## Optimizador de cartera

Este escenario usa el modelo de optimización de ingreso financiero sujeto a condiciones de saturación.

### Condiciones actuales: 
- Días mora es 30, pero cambiar para que sean días mora nivel cliente
- Cliente no es accionista
- Cliente no tuvo cambio de tasa (esto modificarlo). (Sugerencia: aumentar de forma controlada a quienes cumplan alguna condición, como estar debajo de la media)
- Interno RCI 60 y externo 80 para remesa y PV pendiente
- Quitar clientes que tengan inconsistencias en RCI (externo menor al interno)
- Quitar el aumento de clientes que no se les sube más de 0.5%

### Pendientes: 
- Segmentación de clientes según tipo de pago y si cartera es administrable o no
- Mapa de segmentación para cartera administrable (EstadoCredito = "D") según perfiles y su aumento de tasa promedio propuesto

In [1]:
# importar librerías
import pandas as pd
import sys
import os 
import pyodbc
import numpy as np
from datetime import datetime

In [2]:
# importar funciones para el ensyao
sys.path.append(os.path.abspath(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos"))
from utils import calcular_cuota, ingresos_financieros, tasa_minima, tasa_maxima, meses_entre_fechas, ingreso_fin_entre_fechas

In [3]:
def leer_datos(descargar: bool = False):
    if descargar:
        try:
            import pyodbc
            from azure.identity import InteractiveBrowserCredential

            server = 'btprdsynapse-ondemand.sql.azuresynapse.net'
            database = 'OR_MODELOS_INFERIDOS'
            driver = '{ODBC Driver 17 for SQL Server}'
            credential = InteractiveBrowserCredential()
            token = credential.get_token("https://database.windows.net/.default")
            conn_str = f"Driver={driver};Server={server};Database={database};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
            token_bytes = bytes(token.token, 'utf-8')
            exptoken = 'Bearer ' + token_bytes.decode('utf-8')
            conn = pyodbc.connect(conn_str, attrs_before={"Authentication": exptoken})
            # Probar la conexión ejecutando una consulta
            cursor = conn.cursor()
            cursor.execute("SELECT TOP 10 * FROM [silver].[proyeccion_incremento_tasas]")
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        except Exception as e:
            print(e)
    if not descargar:
            cartera = pd.read_csv("../db/cartera 2.csv")
            return cartera

In [7]:
meses_2024 = 12 - datetime.today().month + 1

cartera = leer_datos()
cartera[["SaldoCapital", "CuotaCredito", "Plazo", "MontoDesembolsado",
          "IngresosValidos", "RciGlobalInicial", "Tasa"]].describe()

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_130688\2501476764.py:25: DtypeWarning: Columns (43,45,46,61,63,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cartera = pd.read_csv("../db/cartera 2.csv")


,SaldoCapital,CuotaCredito,Plazo,MontoDesembolsado,IngresosValidos,RciGlobalInicial,Tasa
count,2.920130e+05,2.920130e+05,292013.000000,3.192880e+05,2.823690e+05,282369.000000,292013.000000
mean,8.815110e+04,2.070077e+03,93.899172,9.559004e+04,8.667091e+03,60.068497,19.117845
std,8.629594e+04,4.188683e+03,33.438883,9.233878e+04,4.290676e+04,52.418857,3.881644
min,1.000000e-02,0.000000e+00,2.000000,0.000000e+00,-9.290200e+02,-1283.870000,8.000000
25%,2.185970e+04,7.207200e+02,60.000000,2.500000e+04,4.754160e+03,38.160000,16.300000
50%,5.993545e+04,1.646220e+03,120.000000,6.600000e+04,7.277200e+03,56.280000,18.800000
75%,1.329570e+05,2.936300e+03,120.000000,1.450000e+05,9.319800e+03,74.380000,22.000000
max,2.750000e+06,2.000000e+06,249.000000,3.300000e+06,1.997777e+07,3238.130000,42.000000


In [ ]:
patronos = ["Departamento de Finanzas del Ejercito", "Estado Pasivas", 
            "Instituto Guatemalteco De Seguridad Social Igss", "Ministerio De Cultura Y Deportes Presup",
            "Ministerio de Educacion", "Ministerio de Gobernacion", "Ministerio Publico Presupuesto", 
            "Ministerio Salud Publica Presupuesto", "Organismo Judicial", "Pagos Directos a Caja Privadas",
            "Universidad de san Carlos de Guatemala" ,
            "Secretaria Ejecutiva de la Presidencia de la Republica",
            "Ministerio Comunicaciones y Obras Public"]


cartera = cartera[(cartera["RciGlobalInicial"] > 0) & (cartera["IngresosValidos"] > 0)
                  & (cartera["CuotaCredito"] > 0) & (cartera["MontoDesembolsado"] > 0)
                  & (cartera["SaldoCapital"] > 0) & (cartera["Tasa"] > 0) 
                  & (cartera["EstadoCredito"] == "D") & (cartera["PatronoActual"].isin(patronos))
                  & (cartera["Plazo"] > 0)]


# cartera = cartera[(cartera["CantidadAdelantos"] == 0) & (cartera["Novaciones"] == 0)
#                   & (cartera["Prorroga"] == 0) & (cartera["Reestructuras"] == 0)
#                   & (cartera["Rehabilitaciones"] == 0) & (cartera["Readecuaciones"] == 0)
#                   & (cartera["Convenio"] == 0) & (cartera["Cancelaciones"] == 0)
#                   & (cartera["Amortizaciones"] == 0) & (cartera["Diferimiento"] == 0)]

# Ahora calcular los nuevos RCIs
cartera_temp = cartera.groupby(by="CodigoCliente").agg(
    CUOTA_INTERNA_CLIENTE=("CuotaCredito", "sum"),
    DIAS_MORA_CLIENTE=("DiasMora", "max"),
    MORA_EXTERNA_CLIENTE=("DiasMoraExterna", "max")
)
cartera_temp.columns = ["CUOTA INTERNA CLIENTE", "MORA CLIENTE", "MORA EXTERNA CLIENTE"]
cartera = pd.merge(cartera, cartera_temp, on="CodigoCliente", how="left")

cartera = cartera[(cartera["RciInternoInicial"] > 0) & (cartera["RciInternoInicial"] <= 60)
                  & (cartera["RciGlobalInicial"] <= 80)]
cartera.head(3)

# Sumar la cuota de seguro
def adicional_seguro(monto_desembolsado):
    return 1.12 * monto_desembolsado * 9.4284/12000 + 6.65

cartera["SEGURO"] = cartera.apply(lambda x: adicional_seguro(x["MontoDesembolsado"]), axis=1)

In [ ]:
len(cartera)

In [ ]:
# Leer datos de clientes que tuvieron cambio de tasa
clientes_cambiados = pd.read_excel("../db/Clientes_incremento_tasa_2023.xlsx")
clientes_cambiados.head(3)

# exlcluir accionistas
accionistas = pd.read_csv("../db/CreditosAExcluir.csv", sep=";")
accionistas.head(3)

In [ ]:
cartera.sort_values(by=["DPI", "SaldoCapital"], ascending=[False, False], inplace=True)
cartera.drop_duplicates(subset="DPI", inplace=True)

In [ ]:
codigos_clientes_cambiados = clientes_cambiados["CODIGO_CLIENTE"].unique().tolist()
cartera["TUVO CAMBIO 2023"] = np.where(cartera["CodigoCliente"].isin(codigos_clientes_cambiados), 1, 0)

codigos_accionistas = accionistas["NO_CREDITO"].unique().tolist()
cartera["ES ACCIONISTA"] = np.where(cartera["NoCredito"].isin(codigos_accionistas), 1, 0)

cartera = cartera[~((cartera["TUVO CAMBIO 2023"] == 1) | (cartera["ES ACCIONISTA"] == 1))]
# cartera = cartera[cartera["ES ACCIONISTA"] == 0]

cartera.sort_values(by=["DPI", "SaldoCapital"], ascending=[False, False], inplace=True)
cartera.drop_duplicates(subset="DPI", inplace=True)

In [ ]:
# Calcular categoría interna
def categoria(x):
    if x <= 30:
        return "A"
    if 30 < x <= 90:
        return "B"
    if 90 < x <= 180:
        return "C"
    if 180 < x <= 360:
        return "D"
    else:
        return "E"
    
cartera["CATEGORIA INTERNA"] = cartera.apply(lambda x: categoria(x["DiasMora"]), axis=1)
cartera["CATEGORIA EXTERNA"] = cartera.apply(lambda x: categoria(x["DiasMoraExterna"]), axis=1)

cartera = cartera[(cartera["MORA CLIENTE"] <= 30) & (cartera["MORA EXTERNA CLIENTE"] <= 30)
                  & (cartera["CATEGORIA EXTERNA"] == "A") & (cartera["MesesPendientes"] >= 12 + meses_2024)
                  & (cartera["CATEGORIA INTERNA"] == "A")]

# Modelar Optimizador de cartera
dic_reservas = {"A": 0.036*0.45, "B": 0.086*0.45, "C": 0.156*0.45, "D": 0.325*1, "E": 1}

# Se simulará el precio con la categoría externa para simular contagio
cartera["PCT RESERVAS"] = cartera.apply(lambda x: dic_reservas.get(max(x["CategoriaExterna"], x["CATEGORIA INTERNA"])), axis=1)
cartera["TASA MINIMA"] = cartera.apply(lambda x: tasa_minima(x["PCT RESERVAS"]), axis=1)
cartera["TASA MINIMA"] = 100 * cartera["TASA MINIMA"]

In [ ]:
# Ver estado de RCIs para determinar cotas de no superación
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.hist(cartera["RciInternoInicial"], bins=50)
plt.title("RCI interno inicial")
plt.xlim([0, 100])
plt.show()

In [ ]:
from math import ceil
cartera["COTA RCI"] = cartera.apply(lambda x: min(10*(ceil(x["RciInternoInicial"]/10) + 1), 60), axis=1)
cartera["TASA MAXIMA"] = cartera.apply(lambda x: tasa_maxima(x["RciInternoInicial"], 60), axis=1)

# Ahora evaluar que la tasa máxima no suba tanto
def clasificador(tasa_actual: float, tasa_minima: float, tasa_maxima: float) -> str:
    if tasa_minima <= tasa_maxima:
        if tasa_actual < tasa_minima:
            mensaje = "Cliente estaba por debajo de la mínima"
        elif tasa_actual >= tasa_minima and tasa_actual <= tasa_maxima:
            mensaje = "Cliente apto para intervalo"
        elif tasa_actual > tasa_maxima:
            mensaje = "Cliente saturado: permanecer en sus condiciones"
    elif tasa_minima > tasa_maxima:
        mensaje = "Caso especial"
    return mensaje

cartera["OBSERVACIONES"] = cartera.apply(lambda x: clasificador(x["Tasa"], x["TASA MINIMA"],
                                                                 x["TASA MAXIMA"]), axis=1)

# Ahora sacar la tasa final para los clientes
mascara_debajo_minima = cartera["OBSERVACIONES"] == "Cliente estaba por debajo de la mínima"
mascara_apto = cartera["OBSERVACIONES"] == "Cliente apto para intervalo"
mascara_arriba_max = cartera["OBSERVACIONES"] == "Cliente saturado: permanecer en sus condiciones"
mascara_no_intervalo = cartera["OBSERVACIONES"] == "Caso especial"

###################### CALCULO DE TASA ############
def calcular_tasa(tasa_minima: float, tasa_maxima: float, riesgo: float, tasa_actual: float) -> float:
    tasa = 1.05 * tasa_actual + (tasa_maxima - tasa_minima) * (1001 - riesgo) / 1000
    return tasa

cartera.loc[mascara_apto, "TASA CALCULADA"] = cartera.loc[mascara_apto].apply(lambda x: calcular_tasa(x["TASA MINIMA"], x["TASA MAXIMA"],
                                                                   x["ScoreInstallment"], x["Tasa"]), axis=1)

# Por debajo de la mínima llevarlos a mínima
cartera.loc[mascara_debajo_minima, "TASA SUGERIDA"] = cartera.loc[mascara_debajo_minima, "TASA MINIMA"]

# Correctos, encontrar tasa
cartera.loc[mascara_apto, "TASA SUGERIDA"] = cartera.loc[mascara_apto, "TASA CALCULADA"]

# Arriba de máxima, mantenerlos
cartera.loc[mascara_arriba_max, "TASA SUGERIDA"] = cartera.loc[mascara_arriba_max, "Tasa"]

# Ahora los casos especiales
cartera.loc[mascara_no_intervalo, "TASA SUGERIDA"] = cartera.loc[mascara_no_intervalo].apply(lambda x: max(x["Tasa"], x["TASA MINIMA"]), axis=1)

In [ ]:
# Limpiar la diferencia de precios según tipo de pago
def suavizar_tasa(tipo_pago: int, tasa_sugerida: float, 
                  tasa_actual: float, tuvo_cambio_2023: int) -> float:
    if tuvo_cambio_2023 == 0:
        if tipo_pago == 1:
            tasa = min(tasa_actual + max(0.5, min(tasa_sugerida - tasa_actual, 3.3)), 27)
        if tipo_pago == 0:
            tasa = min(30, tasa_actual + max(0.5, min(tasa_sugerida - tasa_actual, 4)))
    if tuvo_cambio_2023 == 1:
        if tipo_pago == 1:
            tasa = min(27, tasa_actual + max(0.5, min(tasa_sugerida - tasa_actual, 0.5)))
        if tipo_pago == 0:
            tasa = min(30, tasa_actual + max(0.5, min(tasa_sugerida - tasa_actual, 1)))
    return tasa

cartera["TASA SUAVIZADA"] = cartera.apply(lambda x: suavizar_tasa(x["RemesaActual"], x["TASA SUGERIDA"], 
                                                                  x["Tasa"], x["TUVO CAMBIO 2023"]), axis=1)

In [ ]:
cartera["NUEVA CUOTA"] = cartera.apply(lambda x: calcular_cuota(x["MontoDesembolsado"], x["TASA SUAVIZADA"]/1200, x["Plazo"]), axis=1)
cartera["NUEVA CUOTA"] = cartera["NUEVA CUOTA"] + cartera["SEGURO"]
cartera["NUEVO RCI"] = (cartera["CUOTA INTERNA CLIENTE"] - cartera["CuotaCredito"] + cartera["NUEVA CUOTA"]) / cartera["IngresosValidos"] * 100

# Calcular el RCI Global final 
cartera["NUEVO RCI GLOBAL"] = (cartera["IngresosValidos"] * cartera["RciGlobalInicial"] - cartera["CuotaCredito"] + cartera["NUEVA CUOTA"]) / cartera["IngresosValidos"]

# Filtrar a los clientes válidos (que cumplan no pasarse de 60% RCI interno y 80% RCI Global)
cartera = cartera[(cartera["NUEVO RCI"] <= 60) & 
                  ((cartera["NUEVO RCI GLOBAL"] <= 80) & (cartera["RemesaActual"] == 1)
                  | ((cartera["NUEVO RCI GLOBAL"] <= 70) & (cartera["RemesaActual"] == 0)))]

# Agregar el filtro de liquidez
cartera["LIQUIDEZ GLOBAL EN QUETZALES"] = cartera["IngresosValidos"] * (100 - cartera["NUEVO RCI GLOBAL"]) / 100
cartera = cartera[cartera["LIQUIDEZ GLOBAL EN QUETZALES"] >= 200]

# Verificar proporción de aumento de la cuota
cartera["PROPORCION AUMENTO"] = cartera["NUEVA CUOTA"] / cartera["CuotaCredito"] - 1
# cartera = cartera[cartera["PROPORCION AUMENTO"] <= 0.3]
cartera[["RciInternoInicial", "NUEVO RCI"]]

In [ ]:
import matplotlib.pyplot as plt
# Crear la figura para los gráficos 2x2
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Primer gráfico
axes[0, 0].hist(cartera.loc[cartera["RemesaActual"] == 1]["RciInternoInicial"], bins=50, color="#e2056b")
axes[0, 0].set_xlim([0, 65])
axes[0, 0].set_title("RCI interno Remesa inicial")

# Segundo gráfico
axes[0, 1].hist(cartera.loc[cartera["RemesaActual"] == 0]["RciInternoInicial"], bins=50, color="#04b4b3")
axes[0, 1].set_xlim([0, 65])
axes[0, 1].set_title("RCI interno PV inicial")

# Tercer gráfico
axes[1, 0].hist(cartera.loc[cartera["RemesaActual"] == 1]["NUEVO RCI"], bins=50, color="#e2056b")
axes[1, 0].set_xlim([0, 65])
axes[1, 0].set_title("NUEVO RCI Remesa Final")

# Cuarto gráfico
axes[1, 1].hist(cartera.loc[cartera["RemesaActual"] == 0]["NUEVO RCI"], bins=50, color="#04b4b3")
axes[1, 1].set_xlim([0, 65])
axes[1, 1].set_title("RCI interno PV Final")

# Ajustar diseño
plt.tight_layout()
plt.savefig("../db/RCI.png")
plt.show()

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Primer gráfico
axes[0, 0].hist(cartera.loc[cartera["RemesaActual"] == 1]["RciGlobalInicial"], bins=50, color="#e2056b")
axes[0, 0].set_xlim([0, 85])
axes[0, 0].set_title("RCI Global Remesa inicial")

# Segundo gráfico
axes[0, 1].hist(cartera.loc[cartera["RemesaActual"] == 0]["RciGlobalInicial"], bins=50, color="#04b4b3")
axes[0, 1].set_xlim([0, 85])
axes[0, 1].set_title("RCI Global PV inicial")

# Tercer gráfico
axes[1, 0].hist(cartera.loc[cartera["RemesaActual"] == 1]["NUEVO RCI GLOBAL"], bins=50, color="#e2056b")
axes[1, 0].set_xlim([0, 85])
axes[1, 0].set_title("Nuevo RCI Global Remesa Final")

# Cuarto gráfico
axes[1, 1].hist(cartera.loc[cartera["RemesaActual"] == 0]["NUEVO RCI GLOBAL"], bins=50, color="#04b4b3")
axes[1, 1].set_xlim([0, 85])
axes[1, 1].set_title("RCI Global  PV Final")

# Ajustar diseño
plt.tight_layout()
plt.savefig("../db/RCI.png")
plt.show()

In [ ]:
tpp_actual = np.dot(cartera["Tasa"], cartera["MontoDesembolsado"]) / sum(cartera["MontoDesembolsado"])
tpp_nueva = np.dot(cartera["TASA SUAVIZADA"], cartera["MontoDesembolsado"]) / sum(cartera["MontoDesembolsado"])

In [ ]:
tpp_actual

In [ ]:
tpp_nueva

In [ ]:
cartera["RemesaActual"].value_counts()

In [ ]:
cartera["RciInternoInicial"].mean()

In [ ]:
cartera["NUEVO RCI"].mean()

In [ ]:
remesa = cartera[cartera["RemesaActual"] == 0]
np.dot(remesa["MontoDesembolsado"], remesa["Tasa"]) / sum(remesa["MontoDesembolsado"])

In [ ]:
remesa["NUEVO RCI"].mean()

In [ ]:
columnas = cartera.columns.tolist()
columnas.remove("TASA SUGERIDA")
columnas.remove("COTA RCI")
columnas.remove("TASA CALCULADA")
columnas.remove("PCT RESERVAS")

In [ ]:
hoy = datetime.strftime(datetime.today(), format="%d_%m_%Y")
cartera.to_csv(f"../db/Precios cartera/Ensayo cartera_{hoy}.csv")

In [ ]:
cartera["SaldoCapital"].sum() / 10**6

### RESUMEN DE ESCENARIOS

In [ ]:
cartera["INCREMENTO CUOTA"] = cartera["NUEVA CUOTA"] - cartera["CuotaCredito"] 

In [ ]:
mascara_remesa = cartera["RemesaActual"] == 1
pv = cartera.loc[~mascara_remesa]
remesa = cartera.loc[mascara_remesa]
remesa["QUINTIL"] = pd.qcut(remesa["INCREMENTO CUOTA"], 5, labels=[f"QUINTIL {i}" for i in range(1, 6)])
pv["QUINTIL"] = pd.qcut(pv["INCREMENTO CUOTA"], 5, labels=[f"QUINTIL {i}" for i in range(1, 6)])

In [ ]:
impactos = remesa.groupby(by="QUINTIL").agg(
    CLIENTES=("CodigoCliente", "size"),
    SUMA_SALDO=("SaldoCapital", "sum"),
    RCI_INICIAL_INTERNO_PROMEDIO=("RciInternoInicial", "mean"),
    RCI_GLOBAL_INICIAL_PROMEDIO=("RciGlobalInicial", "mean"),
    RCI_INTERNO_FINAL_PROMEDIO=("NUEVO RCI", "mean"),
    RCI_GLOBAL_FINAL_PROMEDIO=("NUEVO RCI GLOBAL", "mean"),
    PROMEDIO_CUOTA_INICIAL=("CuotaCredito", "mean"),
    PROMEDIO_NUEVA_CUOTA=("NUEVA CUOTA", "mean"),
    AUMENTO_CUOTA_PROMEDIO=("INCREMENTO CUOTA", "mean")
).reset_index()

impactos.columns = ["QUINTIL DIFERENCIA AUMENTO CUOTA", "CLIENTES", "SALDO",
                    "RCI INTERNO INICIAL PROMEDIO", "RCI GLOBAL INICIAL PROMEDIO", 
                    "RCI INTERNO FINAL PROMEDIO", "RCI GLOBAL FINAL PROMEDIO", 
                    "PROMEDIO CUOTA INICIAL", "PROMEDIO CUOTA FINAL", 
                    "PROMEDIO AUMENTO CUOTA"]

# Definir nombres cortos para las columnas
impactos.columns = [
    "QUINTIL", "CLIENTES", "SALDO", 
    "RCI INT INI", "RCI GLOB INI", 
    "RCI INT FIN", "RCI GLOB FIN", 
    "CUOTA INI", "CUOTA FIN", "AUM CUOTA"
]

# Formatear todas las columnas numéricas a dos decimales
for col in impactos.columns:
    if impactos[col].dtype in ['float64', 'int64']:  # Verificar que sean numéricas
        impactos[col] = impactos[col].apply(lambda x: f"{x:.2f}")

# Dividir las columnas en dos partes
columns_part1 = ["QUINTIL", "CLIENTES", "SALDO", "RCI INT INI", "RCI GLOB INI"]
columns_part2 = ["RCI INT FIN", "RCI GLOB FIN", "CUOTA INI", "CUOTA FIN", "AUM CUOTA"]

impactos_part1 = impactos[columns_part1]
impactos_part2 = impactos[columns_part2]

# Exportar las tablas en LaTeX
with open("../db/Resumen_impactos.txt", "w") as f:
    # Primera parte de las columnas
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\resizebox{\\textwidth}{!}{%\n")
    f.write(impactos_part1.to_latex(index=False, escape=False))
    f.write("}\n")
    f.write("\\caption{Resumen de Impactos - Parte 1}\n")
    f.write("\\label{tab:resumen_impactos_part1}\n")
    f.write("\\end{table}\n\n")
    
    # Segunda parte de las columnas
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\resizebox{\\textwidth}{!}{%\n")
    f.write(impactos_part2.to_latex(index=False, escape=False))
    f.write("}\n")
    f.write("\\caption{Resumen de Impactos - Parte 2}\n")
    f.write("\\label{tab:resumen_impactos_part2}\n")
    f.write("\\end{table}\n")


In [ ]:
cartera["INGRESOS FINANCIEROS FEBRERO A DIC 2025"] = cartera.apply(lambda x: 
                                                                ingreso_fin_entre_fechas(x["MontoDesembolsado"],
                                                                                         x["TASA SUAVIZADA"] / 1200,
                                                                                         x["Plazo"],
                                                                                         x["MesesTranscurridos"] + 2,
                                                                                         x["MesesTranscurridos"] + 13),
                                                                                           axis=1)


cartera["INGRESOS ORIGINALES FEBRERO A DIC 2025"] = cartera.apply(lambda x: 
                                                                ingreso_fin_entre_fechas(x["MontoDesembolsado"],
                                                                                         x["Tasa"] / 1200,
                                                                                         x["Plazo"],
                                                                                         x["MesesTranscurridos"] + 2,
                                                                                         x["MesesTranscurridos"] + 13),
                                                                                           axis=1)

cartera["DIFERENCIA INGRESO"] = cartera["INGRESOS FINANCIEROS FEBRERO A DIC 2025"] - cartera["INGRESOS ORIGINALES FEBRERO A DIC 2025"]

In [ ]:
clientes = len(cartera)
saldo = cartera["SaldoCapital"].sum() / 10**6
producto_financiero = cartera["INGRESOS FINANCIEROS FEBRERO A DIC 2025"].sum() / 10**6 - cartera["INGRESOS ORIGINALES FEBRERO A DIC 2025"].sum() / 10**6

tpp_todos_final = np.dot(cartera["MontoDesembolsado"], cartera["TASA SUAVIZADA"]) / cartera["MontoDesembolsado"].sum()
tpp_todos_inicial = np.dot(cartera["MontoDesembolsado"], cartera["Tasa"]) / cartera["MontoDesembolsado"].sum()
tpp_remesa_final = np.dot(cartera.loc[mascara_remesa]["MontoDesembolsado"], cartera.loc[mascara_remesa]["TASA SUAVIZADA"]) / cartera.loc[mascara_remesa]["MontoDesembolsado"].sum()
tpp_remesa_inicial = np.dot(cartera.loc[mascara_remesa]["MontoDesembolsado"], cartera.loc[mascara_remesa]["Tasa"]) / cartera.loc[mascara_remesa]["MontoDesembolsado"].sum()
tpp_pv_final = np.dot(cartera.loc[~mascara_remesa]["MontoDesembolsado"], cartera.loc[~mascara_remesa]["TASA SUAVIZADA"]) / cartera.loc[~mascara_remesa]["MontoDesembolsado"].sum()
tpp_pv_inicial = np.dot(cartera.loc[~mascara_remesa]["MontoDesembolsado"], cartera.loc[~mascara_remesa]["Tasa"]) / cartera.loc[~mascara_remesa]["MontoDesembolsado"].sum()

cuota_inicial_promedio = cartera["CuotaCredito"].mean()
cuota_final_promedio = cartera["NUEVA CUOTA"].mean()
cuota_inicial_remesa = cartera.loc[mascara_remesa]["CuotaCredito"].mean()
cuota_final_remesa = cartera.loc[mascara_remesa]["NUEVA CUOTA"].mean()
cuota_inicial_pv = cartera.loc[~mascara_remesa]["CuotaCredito"].mean()
cuota_final_pv = cartera.loc[~mascara_remesa]["NUEVA CUOTA"].mean()

# sacar RCIs
rci_intenro_inicial = cartera["RciInternoInicial"].mean()
rci_interno_final = cartera["NUEVO RCI"].mean()
rci_global_inicial = cartera["RciGlobalInicial"].mean()
rci_global_final = cartera["NUEVO RCI GLOBAL"].mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Nombres de las variables
names = [
    "Créditos", "Saldo (MM)", "Producto Financiero",
    "TPP Todos Final", "TPP Todos Inicial",
    "TPP Remesa Final", "TPP Remesa Inicial",
    "TPP PV Final", "TPP PV Inicial",
    "Cuota Inicial Promedio", "Cuota Final Promedio",
    "Cuota Inicial Remesa", "Cuota Final Remesa",
    "Cuota Inicial PV", "Cuota Final PV",
    "RCI Interno Inicial", "RCI Interno Final",
    "RCI Global Inicial", "RCI Global Final"
]

# Valores correspondientes
values = [
    clientes, saldo, producto_financiero,
    tpp_todos_final, tpp_todos_inicial,
    tpp_remesa_final, tpp_remesa_inicial,
    tpp_pv_final, tpp_pv_inicial,
    cuota_inicial_promedio, cuota_final_promedio,
    cuota_inicial_remesa, cuota_final_remesa,
    cuota_inicial_pv, cuota_final_pv,
    rci_intenro_inicial, rci_interno_final,
    rci_global_inicial, rci_global_final  # Corregido aquí
]

# Crear la cuadrícula 6x3 (para acomodar las 19 variables)
fig, axes = plt.subplots(7, 3, figsize=(15, 16))

# Aplanar los ejes para iterar
axes = axes.flatten()

# Rellenar cada celda con el nombre y valor de la variable
for i, ax in enumerate(axes):
    if i < len(names):
        # Mostrar el nombre de la variable arriba y el valor abajo con diferente tamaño
        ax.text(0.5, 0.7, names[i], fontsize=12, ha='center', va='center', fontweight='bold')
        ax.text(0.5, 0.4, f"{values[i]:,.2f}", fontsize=18, ha='center', va='center', color="blue")
        
        # Activar las líneas divisorias estilo tarjeta
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_linewidth(1.5)
    else:
        ax.axis("off")  # Desactivar las celdas vacías

    # Quitar números de los ejes
    ax.set_xticks([])
    ax.set_yticks([])

# Ajustar espaciado
plt.tight_layout()
plt.show()
